In [1]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
from pyomo.environ import *
import itertools, random, math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# !apt-get install -y -qq coinor-cbc
# opt= SolverFactory('cbc', executable='/usr/bin/cbc')
# import os
# os.environ['NEOS_EMAIL'] = 'XXXXXX@gmail.com'
# from pyomo import environ as pe
# opt = pe.SolverManagerFactory('neos')

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 125081 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [2]:
# ---------------------------
# Dados de exemplo (mude conforme necessário)
# ---------------------------
V = 2                   # número de veículos
T = 20                  # horizonte de tempo (t = 0..T)
lanes = 1               # pistas/lane
Rg = 10                  # granularity for vehicles’ position measurement
P = [1,2]             # plataformas
U = 2                   # number of platforms
K = 2                 # Loading/Unloading time = K * delta_t
rul = {1: [1, 1], 2: [10, 1]} #, 3: [7, 1]} # posições das plataformas (discretas) todas plataformas estão na lane 1

In [3]:
# ---------------------------
# 🔹 Geração estocástica de chegada de carga (Poisson)
# ---------------------------
random.seed(42)
np.random.seed(42)

# taxa média de chegada (λ por ciclo) para cada plataforma
lambda_rate = {1: 1., 2: 1.5, 3: 1.3}

c_u_f_t = {}
for u in P:
    for t in range(T):
        arrivals = np.random.poisson(lambda_rate[u])
        c_u_f_t[(u,t)] = random.choice([x for x in P if x != u]) if arrivals >= 1 else 0

c_u_t_u = {}
for u in P:
    for t in range(T):
      for u2 in P:
        c_u_t_u[(u, t, u2)] = 1 if c_u_f_t[(u,t)] == u2 else 0


#c_u_t2 = {(u, t): int(np.random.poisson(lambda_rate[u]) >= 1) for u in ru for t in range(T)}

# imprimir o mapa de chegadas
print("Mapa de chegadas de carga (1=carga disponível, destino):")
for u in P:
    row = [c_u_f_t[(u,t)] for t in range(10)]
    print(f"  Plataforma {u}: {row}")

# Condições iniciais (posições dos veículos)
# Geração uniforme entre 0 e track_length
l0 = {v: random.randint(1,lanes) for v in range(1, V + 1)}
rvl = {v: [random.randint(1,Rg), l0[v]] for v in range(1, V + 1)}

# imprimir
print("Vehicls possition:")
for v in range(1,V+1):
    print(f"  Vehicle {v}: {rvl[(v)]}")

vLoad = {v: (1, random.choice(P)) if random.uniform(0,1)>0.5 else (0,0) for v in range(1,V+1)}

# imprimir
print("Vehicls cargo:")
for v in range(1,V+1):
    print(f"  Vehicle load {v}: {vLoad[(v)]}")

Mapa de chegadas de carga (1=carga disponível, destino):
  Plataforma 1: [2, 2, 0, 0, 2, 2, 0, 0, 0, 2]
  Plataforma 2: [1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
Vehicls possition:
  Vehicle 1: [4, 1]
  Vehicle 2: [6, 1]
Vehicls cargo:
  Vehicle load 1: (0, 0)
  Vehicle load 2: (0, 0)


In [4]:
# ---------------------------
# Modelo Discreto Pyomo
# ---------------------------
model = ConcreteModel()
# model = AbstractModel()

# Sets
model.V = RangeSet(1, V)
model.Lanes = RangeSet(1, lanes)
model.T = RangeSet(0, T-1)
model.U = RangeSet(1, U)
model.R = RangeSet(1, Rg)
model.K = RangeSet(0, K)

# Construir dicionário completo para Param com valores 0/1
rv = {}
lv = {}
for v in model.V:
    rv[(v)] = rvl[v][0]
    lv[(v)] = rvl[v][1]


# Construir dicionário completo para Param com valores 0/1
ru = {}
lu = {}
for u in model.U:
    ru[(u)] = rul[u][0]
    lu[(u)] = rul[u][1]

# Parâmetros
model.ru = Param(model.U, initialize=ru, domain=Integers)
model.lu = Param(model.U, initialize=lu, domain=Integers)
model.cu = Param(model.U, model.T, model.U, initialize=c_u_t_u, domain=Binary)
# model.dsafe = Param(initialize=dsafe)
# model.delta_t = Param(initialize=delta_t)
# model.delta_s = Param(initialize=delta_s)
# model.sMaxL = Param(initialize=max_s_l)
# model.sMax = Param(initialize=max_s_ul)
# model.L = Param(initialize=L)
# model.R_vals = Param(model.R, within=NonNegativeIntegers, initialize={ ... })

# Variáveis de decisão
model.rv = Var(model.V, model.T, domain=Integers, bounds=(1, Rg))
model.lv = Var(model.V, model.T, domain=Integers, bounds=(1, lanes))
model.ar = Var(model.V, model.T, domain=Integers, bounds=(-1, 1))           # acão discreta
model.al = Var(model.V, model.T, domain=Integers, bounds=(-1, 1))           # acão discreta
model.c = Var(model.V, model.T, model.U, domain=Binary)                       # veiculo possui carga?
model.d = Var(model.V, model.T, model.U, domain=Binary)                     # entrega feita?


model.posBound = Var(model.V, model.T, domain=Binary)
model.posLane = Var(model.V, model.R, model.Lanes, model.T, domain=Binary)
# model.at_platform = Var(model.V, model.T, model.U, domain=Binary)
# Introduce auxiliary binary variable to linearize the product of posLane and c
# pos_c_product[v, t, u] = 1 if vehicle v is at platform u at time t-k (for any k in K) AND vehicle v has a load for platform u at time t-1
model.at_platform_with_load = Var(model.V, model.T, model.U, domain=Binary)

# Initialize variables at t=0
# for v in model.V:
#   for u in model.U:
#     for t in model.K:
#       model.c[v, t, u].fix(0)
for v in model.V:
  model.rv[v, 0].fix(rv[v])
  model.lv[v, 0].fix(lv[v])
  model.ar[v, 0].fix(0)
  model.al[v, 0].fix(0)
  # model.posLane[v, rv[v], lv[v], 0].fix(1)
  if vLoad[v][0] == 1:
    model.c[v, 0, vLoad[v][1]].fix(1)
  else:
    for u in model.U:
      model.c[v, 0, u].fix(0)
  # for k in model.K:
  #   for u in model.U:
  #     model.d[v, k, u].fix(0)

In [5]:
# -----------------------------
# Restrições discreta
# -----------------------------
M = 1000  # valor grande, ajustar conforme contexto/modelo

def pos_lane_to_vector_rule0(m, v, r, l, t):
  return m.rv[v, t] - r <= M * (1 - m.posLane[v, r, l, t])
model.r_l_to_vec_r0 = Constraint(model.V, model.R, model.Lanes, model.T, rule=pos_lane_to_vector_rule0)

def pos_lane_to_vector_rule1(m, v, r, l, t):
  return m.rv[v, t] - r >= -M * (1 - m.posLane[v, r, l, t])
model.r_l_to_vec_r1 = Constraint(model.V, model.R, model.Lanes, model.T, rule=pos_lane_to_vector_rule1)

def pos_lane_to_vector_rule2(m, v, r, l, t):
  return m.lv[v, t] - l <= M * (1 - m.posLane[v, r, l, t])
model.r_l_to_vec_r2 = Constraint(model.V, model.R, model.Lanes, model.T, rule=pos_lane_to_vector_rule2)

def pos_lane_to_vector_rule3(m, v, r, l, t):
  return m.lv[v, t] - l >= -M * (1 - m.posLane[v, r, l, t])
model.r_l_to_vec_r3 = Constraint(model.V, model.R, model.Lanes, model.T, rule=pos_lane_to_vector_rule3)

def one_hot_R_rule(m, v, t):
    return sum(m.posLane[v, r, l, t] for r in m.R for l in m.Lanes) == 1
model.one_hot_r = Constraint(model.V, model.T, rule=one_hot_R_rule)

def one_hot_V_rule(m, r, l, t):
    return sum(m.posLane[v, r, l, t] for v in m.V) <= 1
model.one_hot_v = Constraint(model.R, model.Lanes, model.T, rule=one_hot_V_rule)

# Atualização da posição:
def position_rule(m, v, t):
    if t == 0:
        return Constraint.Skip
    return m.rv[v, t] == m.rv[v, t-1] + m.ar[v, t]
model.pos_update = Constraint(model.V, model.T, rule=position_rule)

# Atualização da lane:
def lane_rule(m, v, t):
    if t == 0:
        return Constraint.Skip
    return m.lv[v, t] == m.lv[v, t-1] + m.al[v, t]
model.lane_update = Constraint(model.V, model.T, rule=lane_rule)

# Linearization constraints for at_platform_with_load[v, t, u]
def at_platform_with_load_rule1(m, v, t, u):
    # If vehicle v is at platform u at any time in [t-K, t] AND has load for u at t-1, then at_platform_with_load[v, t, u] can be 1
    return m.at_platform_with_load[v, t, u] <= m.c[v, t, u]
model.at_platform_with_load_1 = Constraint(model.V, model.T, model.U, rule=at_platform_with_load_rule1)

def at_platform_with_load_rule2(m, v, t, u):
    if t == 0:
        return Constraint.Skip
    # If vehicle v is at platform u at any time in [t-K, t] AND has load for u at t-1, then at_platform_with_load[v, t, u] can be 1
    return m.at_platform_with_load[v, t, u] <= m.posLane[v, m.ru[u], m.lu[u], t]
model.at_platform_with_load_2 = Constraint(model.V, model.T, model.U, rule=at_platform_with_load_rule2)

def at_platform_with_load_rule3(m, v, t, u):
    if t == 0:
        return Constraint.Skip
    # If at_platform_with_load[v, t, u] is 1, then vehicle v must be at platform u at some time in [t-K, t] AND have load for u at t-1
    return m.at_platform_with_load[v, t, u] >= m.c[v, t, u] + m.posLane[v, m.ru[u], m.lu[u], t] - 1
model.at_platform_with_load_3 = Constraint(model.V, model.T, model.U, rule=at_platform_with_load_rule3)


def delivery_possible_rule(m, v, t, u):
    if t == 0:
        return Constraint.Skip
    # Linearized delivery constraint using the auxiliary variable
    return m.d[v, t, u] <= m.at_platform_with_load[v, t, u]
model.delivery = Constraint(model.V, model.T, model.U, rule=delivery_possible_rule)

def load_possible_rule_delivery(m, v, t, u):
  if t == 0:
        return Constraint.Skip
    # If a delivery is made, the vehicle should not have a load for this platform at time t
  return m.c[v, t, u] <= 1 - m.d[v, t-1, u]
model.load_constraint_delivery = Constraint(model.V, model.T, model.U, rule=load_possible_rule_delivery)

def load_possible_rule_no_change(m, v, t, u):
    if t == 0:
        return Constraint.Skip
    # If no delivery is made and no loading is possible, the load status remains the same
    # at_platform_sum = sum(m.posLane[v, m.ru[u], m.lu[u], t-k] for k in m.K)
    return m.c[v, t, u] >= m.c[v, t-1, u] - m.posLane[v, m.ru[u], m.lu[u], t]
model.load_constraint_no_change = Constraint(model.V, model.T, model.U, rule=load_possible_rule_no_change)

def load_possible_rule_no_change1(m, v, t, u):
    if t == 0:
        return Constraint.Skip
    # If no delivery is made and no loading is possible, the load status remains the same
    # at_platform_sum = sum(m.posLane[v, m.ru[u], m.lu[u], t-k] for k in m.K)
    exp = sum(m.posLane[v, m.ru[u2], m.lu[u2], t] * m.cu[u2, t, u] for u2 in model.U)
    return m.c[v, t, u] <= m.c[v, t-1, u] + exp
model.load_constraint_no_change1 = Constraint(model.V, model.T, model.U, rule=load_possible_rule_no_change1)

In [6]:
# -----------------------------
# Função objetivo
# -----------------------------
def objective_rule0(m):
    return sum(m.d[v, t, u] for v in m.V for t in m.T for u in m.U)
model.obj0 = Objective(rule=objective_rule0, sense=maximize)
# def objective_rule1(m):
#     return sum(m.posLane[v, r, l, t] for v in m.V for r in m.R for l in m.Lanes for t in m.T)
# model.obj1 = Objective(rule=objective_rule1, sense=maximize)

In [ ]:
model.rv

In [7]:
# -----------------------------
# Solver
# -----------------------------
# solver = SolverFactory("glpk")
# results = solver.solve(model, tee=True)

solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
results = solver.solve(model, tee=True)

# -----------------------------
# Resultados
# -----------------------------
print("\n=== RESULTADOS ===")

print(results.solver.status)
print(results.solver.termination_condition)

# for v in model.V:
#     print(f"Veículo {v}:")
#     for t in model.T:
#         print(f" t={t:2d} | pos={value(model.rv[v,t]):6.2f} | vel={value(model.ar[v,t]):6.2f} | carga={int(value(model.c[v,t]))} | entrega={int(value(model.d[v,t]))}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp1ckka9my.glpk.raw --wglp /tmp/tmpl87otix1.glpk.glp --cpxlp
 /tmp/tmp7xbailno.pyomo.lp
Reading problem data from '/tmp/tmp7xbailno.pyomo.lp'...
/tmp/tmp7xbailno.pyomo.lp:13787: warning: lower bound of variable 'x2' redefined
/tmp/tmp7xbailno.pyomo.lp:13787: warning: upper bound of variable 'x2' redefined
2452 rows, 788 columns, 5400 non-zeros
788 integer variables, 636 of which are binary
14423 lines were read
Writing problem data to '/tmp/tmpl87otix1.glpk.glp'...
11255 lines were written
GLPK Integer Optimizer 5.0
2452 rows, 788 columns, 5400 non-zeros
788 integer variables, 636 of which are binary
Preprocessing...
143 hidden covering inequaliti(es) were detected
616 constraint coefficient(s) were reduced
1295 rows, 599 columns, 3005 non-zeros
599 integer variables, 523 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  9.000e+00  ratio =  9.000e+00
Problem data seem to be wel

In [8]:
results = solver.solve(model, tee=True)
results.write()  # para ver status
print(results.solver.status)
print(results.solver.termination_condition)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpesvhcx6c.glpk.raw --wglp /tmp/tmpphwv6jhf.glpk.glp --cpxlp
 /tmp/tmpyuo55aw7.pyomo.lp
Reading problem data from '/tmp/tmpyuo55aw7.pyomo.lp'...
/tmp/tmpyuo55aw7.pyomo.lp:13787: warning: lower bound of variable 'x2' redefined
/tmp/tmpyuo55aw7.pyomo.lp:13787: warning: upper bound of variable 'x2' redefined
2452 rows, 788 columns, 5400 non-zeros
788 integer variables, 636 of which are binary
14423 lines were read
Writing problem data to '/tmp/tmpphwv6jhf.glpk.glp'...
11255 lines were written
GLPK Integer Optimizer 5.0
2452 rows, 788 columns, 5400 non-zeros
788 integer variables, 636 of which are binary
Preprocessing...
143 hidden covering inequaliti(es) were detected
616 constraint coefficient(s) were reduced
1295 rows, 599 columns, 3005 non-zeros
599 integer variables, 523 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  9.000e+00  ratio =  9.000e+00
Problem data seem to be wel